## Setting

In [1]:
import pandas as pd
import sys
import os
import numpy as np
from clust.pipeline import data_pipeline 

sys.path.append("../")
sys.path.append("../..")
sys.path.append("../../..")
sys.path.append("../../../..")

In [2]:
from clust.ingestion import influx_client

clust_config = {
        "url":'http://10.252.219.52:8086/',
        "token":"lRO_RjCuIpdgzpwx9bwhU_0MhyFflFA_fcPRVy6nGLEmJPnFAjB4zt-xbUh6UCaAA_36jCpZ3s9KCj9KkggcRA==",
        "org":"clust"
}

cedp_config = {
        "url":'http://10.252.219.52:8086/',
        "token":"U9QaDrozQDsGdnxpZkkq5NJ1fwzDefEDtR7a_vAaylcxR6LsY1wCc36QVVqDah6ytctNYFM__CfzFuzQ8K4Niw==",
        "org":"cedp"
}

agridata_config = {
        "url":'http://10.252.219.52:8086/',
        "token":"jUOMSF1jfAg7Kh-DscoswyBfMe6VAN1XdEabpYSTDbkeMw8SdNf1KFmQD0bomJC2rmLy0dwvwLrnySZxCfoMUQ==",
        "org":"cedp"
}

db_client = influx_client.InfluxClient(cedp_config)

## Get Data

In [3]:
bk_name = 'farm_protectedHorticulture_vegetables_cucumber'
ms_name = 'environment_knu_sghDG'

In [4]:
data = db_client.get_data(bk_name,ms_name)
data

,INT_CO2,INT_REL_HD,MID_INT_TMP,SLR_RAD
time,,,,
2021-10-19 00:00:00+00:00,NaN,60.500000,19.800000,0.000000
2021-10-19 00:03:00+00:00,NaN,60.300000,61.724860,0.000000
2021-10-19 00:04:00+00:00,949.695615,NaN,65.942175,0.000000
2021-10-19 00:05:00+00:00,549.000000,147.403441,NaN,0.000000
2021-10-19 00:05:45+00:00,1043.913556,162.209287,19.800000,NaN
...,...,...,...,...
2022-05-02 23:55:00+00:00,503.000000,162.928453,14.700000,0.000000
2022-05-02 23:56:00+00:00,NaN,79.100000,49.295213,0.000000
2022-05-02 23:57:00+00:00,281.965362,167.546283,NaN,0.000000


## Set Parameter

In [5]:
# 측정 기준 정하기
range_limits = {
    'max_num': {
        'INT_CO2': 870, 'INT_REL_HD': 105, 'MID_INT_TMP': 40,'SLR_RAD': 900
    },
    'min_num': {
        'INT_CO2': 290, 'INT_REL_HD': 15, 'MID_INT_TMP': 9,'SLR_RAD': 0
    }
}

expected_types = {'INT_CO2': int, 'INT_REL_HD': float, 'MID_INT_TMP': float, 'SLR_RAD' : float}

error_values = {
    "INT_CO2": [9999, -9999],  
    "MID_INT_TMP": [9999, -9999],  
}

## Test

In [6]:
pipeline_certain = [
      ['data_outlier',
        {'certain_error_to_NaN': {'data_min_max_limit': range_limits, 'flag': True},
         'uncertain_error_to_NaN': {'flag': None,
             'param': {'outlierDetectorConfig': 
                       [{'algorithm': 'IQR',
                            'percentile': 95,
                            'alg_parameter': {'weight':80}
                        }]}}}]
     ]
certain_data = data_pipeline.pipeline(data, pipeline_certain, expected_types=expected_types)

data_outlier
{'certain_error_to_NaN': {'data_min_max_limit': {'max_num': {'INT_CO2': 870, 'INT_REL_HD': 105, 'MID_INT_TMP': 40, 'SLR_RAD': 900}, 'min_num': {'INT_CO2': 290, 'INT_REL_HD': 15, 'MID_INT_TMP': 9, 'SLR_RAD': 0}}, 'flag': True}, 'uncertain_error_to_NaN': {'flag': None, 'param': {'outlierDetectorConfig': [{'algorithm': 'IQR', 'percentile': 95, 'alg_parameter': {'weight': 80}}]}}}
get_errorToNaNData
(280748, 4)


In [7]:
certain_data = certain_data.replace({pd.NA: np.nan})
certain_data[certain_data.columns] = certain_data[certain_data.columns].astype("float64")
certain_data

,INT_CO2,INT_REL_HD,MID_INT_TMP,SLR_RAD
time,,,,
2021-10-19 00:00:00+00:00,NaN,60.5,19.8,0.000000
2021-10-19 00:03:00+00:00,NaN,60.3,NaN,0.000000
2021-10-19 00:04:00+00:00,NaN,NaN,NaN,0.000000
2021-10-19 00:05:00+00:00,549.0,NaN,NaN,0.000000
2021-10-19 00:05:45+00:00,NaN,NaN,19.8,NaN
...,...,...,...,...
2022-05-02 23:55:00+00:00,503.0,NaN,14.7,0.000000
2022-05-02 23:56:00+00:00,NaN,79.1,NaN,0.000000
2022-05-02 23:57:00+00:00,NaN,NaN,NaN,0.000000
